In [1]:
import pandas as pd

In [2]:
final = pd.DataFrame()
for year in range(2013, 2018):
    df = pd.read_excel(f'../data/Drug Utilization Report - {year}/Drug Utilization Report Data - {year}.xlsx', sheet_name=3)
    df['Year'] = year
    df['DEA DRUG SCHEDULE'] = df['DEA DRUG SCHEDULE'].astype(int, errors='ignore')
    df.rename(columns={'DEA DRUG SCHEDULE':'DEA_DRUG_SCHEDULE', 
                       'PRESCRIPTION COUNT (#)':'PRESCRIPTION_COUNT', 
                       'PRESCRIPTION QUANTITY (#)': 'PRESCRIPTION_QUANTITY',
                       'PATIENT COUNTY':'PATIENT_COUNTY',
                       'AHFS DESCRIPTION':'AHFS_DESCRIPTION',
                       'PATIENT STATE':'PATIENT_STATE',
                       'DRUG NAME/STRENGTH':'DRUG_NAME_STRENGTH'}, inplace=True)
    final = pd.concat([final, df], ignore_index=True)

In [3]:
final_age = pd.DataFrame()
for year in range(2018, 2023):
    if year == 2019:
        df = pd.read_excel(f'../data/Drug Utilization Report - {year}/{year}_Michigan_Drug_Utilization_Report_FINAL.xlsb', sheet_name='Patient County')
    elif year >= 2021:
        df = pd.read_excel(f'../data/Drug Utilization Report - {year}/{year}_Michigan_Drug_Utilization_Report_FINAL.xlsb', sheet_name='Patient Zip and County')
        df.rename(columns={'PATIENT ZIP':'PATIENT_ZIP'}, inplace=True)
    else:
        df = pd.read_excel(f'../data/Drug Utilization Report - {year}/{year}_Michigan_Drug_Utilization_Report_FINAL.xlsx', sheet_name='Patient County')

    df['DRUG SCHEDULE'] = df['DRUG SCHEDULE'].str.split().str[-1].astype(int, errors='ignore')
    df['Year'] = year
    df.rename(columns={'DRUG SCHEDULE':'DEA_DRUG_SCHEDULE', 
                    'PRESCRIPTION COUNT':'PRESCRIPTION_COUNT', 
                    'PRESCRIPTION QUANTITY (DOSAGE UNITS)':'PRESCRIPTION_QUANTITY',
                    'PATIENT COUNTY':'PATIENT_COUNTY',
                    'PATIENT STATE':'PATIENT_STATE',
                    'PATIENT COUNT':'PATIENT_COUNT',
                    'AHFS DESCRIPTION':'AHFS_DESCRIPTION',
                    'DRUG NAME/STRENGTH':'DRUG_NAME_STRENGTH',
                    'AGE RANGE':'AGE_RANGE',
                    'AVERAGE DAYS SUPPLY':'AVERAGE_DAYS_SUPPLY',
                    'DAYS SUPPLY':'DAYS_SUPPLY',
                    'AVERAGE DAILY MMEs (*ONLY CALCULATED FOR OPIATE AGONISTS AND OPIATE PARTIAL AGONISTS)': 'AVERAGE_DAILY_MMEs',
                    'PRESCRIPTION COUNT GREATER THAN OR EQUAL TO 90 MMEs (*ONLY CALCULATED FOR OPIATE AGONISTS AND OPIATE PARTIAL AGONISTS)': 'PRESCRIPTION_COUNT_GREATER_THAN_OR_EQUAL_TO_90_MMEs',
                    }, inplace=True)
    agg_df = df.groupby(['PATIENT_COUNTY', 'PATIENT_STATE', 'DRUG_NAME_STRENGTH']).agg({'DEA_DRUG_SCHEDULE': 'first', 'AHFS_DESCRIPTION':'first', 'PRESCRIPTION_COUNT': 'sum', 'PRESCRIPTION_QUANTITY': 'sum', 'Year':'first'}).reset_index()
    agg_df['DEA_DRUG_SCHEDULE'] = agg_df['DEA_DRUG_SCHEDULE'].astype(int, errors='ignore')
    final = pd.concat([final, agg_df], ignore_index=True)
    final_age = pd.concat([final_age, df], ignore_index=True)

In [4]:
final = final.loc[~final['PATIENT_COUNTY'].isna()]
final_age = final_age.loc[~final_age['PATIENT_COUNTY'].isna()]

In [5]:
import sqlite3
conn = sqlite3.connect('../data/michigan_drug.db')
final.to_sql('patient', conn, if_exists='replace', index=False)
final_age.to_sql('patient_with_age', conn, if_exists='replace', index=False)

1879759

In [6]:
conn.commit()
conn.close()

In [7]:
final_age.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1879759 entries, 0 to 1879762
Data columns (total 15 columns):
 #   Column                                               Dtype  
---  ------                                               -----  
 0   PATIENT_COUNTY                                       object 
 1   PATIENT_STATE                                        object 
 2   AGE_RANGE                                            object 
 3   DRUG_NAME_STRENGTH                                   object 
 4   DEA_DRUG_SCHEDULE                                    object 
 5   AHFS_DESCRIPTION                                     object 
 6   PRESCRIPTION_COUNT                                   int64  
 7   PRESCRIPTION_QUANTITY                                float64
 8   PATIENT_COUNT                                        float64
 9   AVERAGE_DAYS_SUPPLY                                  float64
 10  DAYS_SUPPLY                                          float64
 11  AVERAGE_DAILY_MMEs           

In [12]:
import sqlite3
conn = sqlite3.connect('../data/michigan_drug.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
schema = {}
for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    schema[table_name] = [col[1] for col in columns]
print(schema)
conn.close()

{'patient': ['PATIENT_COUNTY', 'PATIENT_STATE', 'DRUG_NAME_STRENGTH', 'DEA_DRUG_SCHEDULE', 'AHFS_DESCRIPTION', 'PRESCRIPTION_COUNT', 'PRESCRIPTION_QUANTITY', 'Year'], 'patient_with_age': ['PATIENT_COUNTY', 'PATIENT_STATE', 'AGE_RANGE', 'DRUG_NAME_STRENGTH', 'DEA_DRUG_SCHEDULE', 'AHFS_DESCRIPTION', 'PRESCRIPTION_COUNT', 'PRESCRIPTION_QUANTITY', 'PATIENT_COUNT', 'AVERAGE_DAYS_SUPPLY', 'DAYS_SUPPLY', 'AVERAGE_DAILY_MMEs', 'PRESCRIPTION_COUNT_GREATER_THAN_OR_EQUAL_TO_90_MMEs', 'Year', 'PATIENT_ZIP']}
